In [55]:

import torch
import torch.nn as nn
import numpy as np
import csv
from pandas.io.parsers import read_csv
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
import torch.nn.functional as F

#dataes =  open('/content/test_data.csv','r')
#dataes = csv.reader(dataes)
#train_data = []

#for data in dataes:
  #train_data.append([data[5],data[8],data[10]])
#print(train_data[1:])



data = read_csv('/content/drive/MyDrive/test03.csv',sep=',',thousands=',' )

xy=np.array(data,dtype=np.float32)

# print(data[5][1:][1].replace(',',''))

#print(float(data[1][2]))
#print(data[1:][5],data[1:][8],data[1:][10])
#data = [data[1:][5],data[1:][8],data[1:][10]]


In [56]:
print(data[0:])

     전용면적(㎡)    계약년월  계약일   층  건축년도  거래금액(만원)
0    70.9700  202208   25   9  2017     65500
1    55.9200  202208    8  15  2015     26900
2   100.4300  202208   19  17  2020    140000
3    73.6700  202208   29   8  2020     77900
4    50.7624  202208   25  13  2013     28000
5    28.9500  202208    6   7  2016     13600
6    60.5900  202208    3   5  2002     23000
7    76.5000  202208    4   5  2003     24800
8    72.0050  202208    8   6  2004     26000
9    59.7600  202208    4   7  1996     29990
10  112.8180  202208   27  13  2005     78000
11   27.3600  202208    4  13  2004      7850
12   23.0400  202208   22  20  2004      7900
13   84.9870  202208   25   4  2005     67000
14   51.5883  202208    6   8  2017     25000
15   43.3300  202208    2   7  2013     13500
16   21.6400  202208    2   6  2013      6500
17   84.4885  202208   22  10  2020     58960
18   79.0200  202208   26   3  1987     99500
19   73.9200  202208    8   1  1979    107000
20   61.0900  202208   13   6  197

In [57]:
print(xy)

[[7.097000e+01 2.022080e+05 2.500000e+01 9.000000e+00 2.017000e+03
  6.550000e+04]
 [5.592000e+01 2.022080e+05 8.000000e+00 1.500000e+01 2.015000e+03
  2.690000e+04]
 [1.004300e+02 2.022080e+05 1.900000e+01 1.700000e+01 2.020000e+03
  1.400000e+05]
 [7.367000e+01 2.022080e+05 2.900000e+01 8.000000e+00 2.020000e+03
  7.790000e+04]
 [5.076240e+01 2.022080e+05 2.500000e+01 1.300000e+01 2.013000e+03
  2.800000e+04]
 [2.895000e+01 2.022080e+05 6.000000e+00 7.000000e+00 2.016000e+03
  1.360000e+04]
 [6.059000e+01 2.022080e+05 3.000000e+00 5.000000e+00 2.002000e+03
  2.300000e+04]
 [7.650000e+01 2.022080e+05 4.000000e+00 5.000000e+00 2.003000e+03
  2.480000e+04]
 [7.200500e+01 2.022080e+05 8.000000e+00 6.000000e+00 2.004000e+03
  2.600000e+04]
 [5.976000e+01 2.022080e+05 4.000000e+00 7.000000e+00 1.996000e+03
  2.999000e+04]
 [1.128180e+02 2.022080e+05 2.700000e+01 1.300000e+01 2.005000e+03
  7.800000e+04]
 [2.736000e+01 2.022080e+05 4.000000e+00 1.300000e+01 2.004000e+03
  7.850000e+03]
 [2.

In [58]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device =='cuda':
  torch.cuda.manual_seed_all(777)

In [59]:

training_epochs = 100


In [60]:
# 4개의 변인을 입력을 받습니다.
# avgTemp,minTemp,maxTemp,rainFall
x_data =torch.FloatTensor(xy[:,0:-1])


# 가격 값을 입력 받습니다.
# avgPrice
y_data =torch.FloatTensor(xy[:,[-1]])
dataset = TensorDataset(x_data,y_data)
data_loader = torch.utils.data.DataLoader(dataset,batch_size = 1,shuffle=True,drop_last=True)



In [61]:
# nn layers
linear1 = torch.nn.Linear(5,1,bias=True)


relu = torch.nn.ReLU()

# xavier initialization#
torch.nn.init.xavier_uniform_(linear1.weight)




Parameter containing:
tensor([[ 0.4485,  0.0065,  0.6497, -0.4731, -0.3775]], requires_grad=True)

In [62]:
# model 
model = torch.nn.Sequential(linear1,relu).to(device)

In [63]:
# define cost/loss & optimizer
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [64]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        #print(X,Y)
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        n=len(xy)
        hypothesis = model(X)
        #print(hypothesis)
        cost = F.mse_loss(hypothesis,Y)


        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 1924770688.000000000
Epoch: 0002 cost = 1287355008.000000000
Epoch: 0003 cost = 1293319552.000000000
Epoch: 0004 cost = 1286227200.000000000
Epoch: 0005 cost = 1282249472.000000000
Epoch: 0006 cost = 1258417536.000000000
Epoch: 0007 cost = 1278318592.000000000
Epoch: 0008 cost = 1293795840.000000000
Epoch: 0009 cost = 1287275008.000000000
Epoch: 0010 cost = 1291671552.000000000
Epoch: 0011 cost = 1257786240.000000000
Epoch: 0012 cost = 1279563776.000000000
Epoch: 0013 cost = 1271743616.000000000
Epoch: 0014 cost = 1297723264.000000000
Epoch: 0015 cost = 1314477696.000000000
Epoch: 0016 cost = 1295598464.000000000
Epoch: 0017 cost = 1362978048.000000000
Epoch: 0018 cost = 1303181056.000000000
Epoch: 0019 cost = 1299439104.000000000
Epoch: 0020 cost = 1319419008.000000000
Epoch: 0021 cost = 1266754688.000000000
Epoch: 0022 cost = 1276658048.000000000
Epoch: 0023 cost = 1278288000.000000000
Epoch: 0024 cost = 1278516352.000000000
Epoch: 0025 cost = 1281754496.000000000


In [65]:
test_data = read_csv('/content/test02.csv',sep=',',thousands=',' )

test_xy = np.array(test_data,dtype=np.float32)
x_data_test = torch.FloatTensor(test_xy[:,0:-1])


In [66]:
# 학습한 W를 train data에 적용해봄

y_pred = model(x_data_test)
for i in range(43):
  print('예측된 가격:',int(y_pred[i].item()))

예측된 가격: 53044
예측된 가격: 52802
예측된 가격: 53330
예측된 가격: 53095
예측된 가격: 52876
예측된 가격: 52474
예측된 가격: 52743
예측된 가격: 52904
예측된 가격: 52899
예측된 가격: 52757
예측된 가격: 53489
예측된 가격: 52484
예측된 가격: 52633
예측된 가격: 53147
예측된 가격: 52698
예측된 가격: 52581
예측된 가격: 52366
예측된 가격: 53155
예측된 가격: 53094
예측된 가격: 52888
예측된 가격: 52838
예측된 가격: 54182
예측된 가격: 53548
예측된 가격: 53136
예측된 가격: 52916
예측된 가격: 53539
예측된 가격: 52808
예측된 가격: 52718
예측된 가격: 53182
예측된 가격: 53107
예측된 가격: 53105
예측된 가격: 53168
예측된 가격: 53025
예측된 가격: 52656
예측된 가격: 52930
예측된 가격: 52830
예측된 가격: 53339
예측된 가격: 53996
예측된 가격: 53201
예측된 가격: 52754
예측된 가격: 53132
예측된 가격: 53033
예측된 가격: 52988
